# Используемые функции

In [ ]:
#скрипт заполнения столбца: (число кликов на документ / количество показов)
from collections import Counter
List_of_train_items = []#d1
List_of_train_items = train_df.item
List_of_test_items = [] #d2
CTR_list_train = [] #l
CTR_list_test = []#l1

Counter_list_of_train_items = Counter(List_of_train_items)#c1
for n in range(train_count):
    if train_df['target'][n]==1:
      List_of_test_items.append(train_df['item'][n])
Counter_list_of_test_items = Counter(List_of_test_items)#c2

for n in range(train_count):
    CTR_list_train.append(Counter_list_of_test_items.get(train_df['item'][n])
                                      /Counter_list_of_train_items.get(train_df['item'][n]))
for n in range(test_count):
    CTR_list_test.append(Counter_list_of_test_items.get(test_df['item'][n])
                                     /Counter_list_of_train_items.get(test_df['item'][n]))
MyFile=open('output_train.txt', 'w')
for element in CTR_list_train:
     MyFile.write(str(round(element*1000000)))
     MyFile.write('\n')
MyFile.close()

MyFile2=open('output_test.txt', 'w')
for element in CTR_list_test:
     MyFile2.write(str(round(element*1000000)))
     MyFile2.write('\n')
MyFile2.close()

In [3]:
from datetime import datetime
import math  
from sklearn.metrics import accuracy_score
import time
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
EPS = 1e-4
def score(logloss):
    if logloss > 0.5:
        return 0.0 
    return min(150, (200 * (0.5 - logloss))** 2)

def logloss(y_true, y_pred):
    if abs(y_pred - 1) < EPS:
        y_pred = 1 - EPS
    if abs(y_pred) < EPS:
        y_pred = EPS
    return(-(y_true * np.log(y_pred)) - (1 - y_true) * np.log(1 - y_pred))

# Исходный датасет 
Исходные данные имеют вид, подходящий для обучения модели, разделим данные на обучающие и тестовые функцией train_test_split. Причем выберем процент тестовой выборки укажем 30%:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.3, random_state = 0)


In [ ]:
import pandas as pd
train_df = pd.read_csv('train.csv', sep=';', engine='python')
test_df = pd.read_csv('test.csv')
train_count = 1009108
test_count = 112124
y = train_df.target
X = train_df.drop('target', axis=1)

cat_features = range(0, X.shape[1])
print(cat_features)

from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.3, random_state=1234)

In [3]:
train_df.head()

,sample_id,item,publisher,user,topic_0,topic_1,topic_2,topic_3,topic_4,weight_0,weight_1,weight_2,weight_3,weight_4,target
0,0,531,147,2925,411,477,618,249,460,27,18,9,8,7,0
1,1,1574,260,2981,212,287,382,302,51,27,11,2,1,0,0
2,2,940,394,1230,145,150,212,170,174,7,6,6,5,5,0
3,3,52,520,2597,201,283,618,249,617,35,33,30,11,9,1
4,4,766,55,1680,362,150,477,305,388,51,15,13,10,9,1


# Засекаем время выполнения скрипта #1

In [ ]:
start_time = datetime.now()
from catboost import CatBoostRegressor
model1 = CatBoostRegressor(
    eval_metric='AUC',
    learning_rate=0.8,
    iterations=500,
    random_seed=42,
    od_type='Iter',
    od_wait=20
)
model1.fit(
    X_train, y_train,
    eval_set=(X_validation, y_validation),
    cat_features=cat_features,
    logging_level='Silent',
    plot=True
)
prediction = model1.predict(X) # передача тестовой выборки в модель
skript_time1 = datetime.now() - start_time
mae = 0
mse = 0
ss_tot = 0
MSLp1E = 0
mean_true = sum(train_df['target'])/train_count
res = 0
for n in range(train_count):
    if train_df['target'][n]==0:
        mae = mae + abs(train_df['target'][n] - prediction[n][0])
        mse = mse + (train_df['target'][n] - prediction[n][0])**2
        ss_tot = ss_tot + (prediction[n][0] - mean_true)**2
        MSLp1E = MSLp1E + (np.log1p(prediction[n][0]) - np.log1p(train_df['target'][n])) ** 2
        if prediction[n][0]>=0.5:
            res = res + 1
    else:
        mae = mae + abs(train_df['target'][n] - prediction[n][1])
        mse = mse + (train_df['target'][n] - prediction[n][1])**2
        ss_tot = ss_tot + (prediction[n][1] - mean_true)**2
        MSLp1E = MSLp1E + (np.log1p(prediction[n][1]) - np.log1p(train_df['target'][n])) ** 2
        if prediction[n][1]>=0.5:
            res = res + 1
res=res/train_count
print('1.1           ', res)
r2_score = 1 - (mse/ train_count)/(ss_tot / train_count)
MSLp1E = MSLp1E / train_count
mse = mse / train_count 
MedAE = np.median(mae)
mae = mae / train_count

In [5]:
print('script 1 working time ', skript_time1, 'mae  ', mae, 'mse ', mse, 'r2_score ', r2_score, 'MedAE ', MedAE, 'MSLp1E ', MSLp1E)

script 1 working time  0:01:53.911205 mae   0.7986315798317545 mse  0.6534059898067504 r2_score  -1.0374905646463848 MedAE  805905.5162608621 MSLp1E  0.33013467560547066


# Засекаем время выполнения скрипта #2

In [ ]:
start_time = datetime.now()
from catboost import CatBoostClassifier
model2 = CatBoostClassifier(
    thread_count=2,
    iterations=300,
    learning_rate=0.1,
    random_seed=63,
    custom_loss=['AUC', 'Accuracy'],
    use_best_model=True
)
model2.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
    logging_level='Silent',
    plot=True
)
prediction = model2.predict_proba(X) # передача тестовой выборки в модель
skript_time2 = datetime.now() - start_time
mae2 = 0
mse2 = 0
ss_tot2 = 0
MSLp1E2 = 0
mean_true2 = sum(train_df['target'])/train_count
res = 0
for n in range(train_count):
    if train_df['target'][n]==0:
        mae2 = mae2 + abs(train_df['target'][n] - prediction[n][0])
        mse2 = mse2 + (train_df['target'][n] - prediction[n][0])**2
        ss_tot2 = ss_tot2 + (prediction[n][0] - mean_true2)**2
        MSLp1E2 = MSLp1E2 + (np.log1p(prediction[n][0]) - np.log1p(train_df['target'][n])) ** 2
        if prediction[n][0]>=0.5:
            res = res + 1
    else:
        mae2 = mae2 + abs(train_df['target'][n] - prediction[n][1])
        mse2 = mse2 + (train_df['target'][n] - prediction[n][1])**2
        ss_tot2 = ss_tot2 + (prediction[n][1] - mean_true2)**2
        MSLp1E2 = MSLp1E2 + (np.log1p(prediction[n][1]) - np.log1p(train_df['target'][n])) ** 2
        if prediction[n][1]>=0.5:
            res = res + 1

res=res/train_count
print('1.2      ',res)
r2_score2 = 1 - (mse2 / train_count)/(ss_tot2 / train_count)
MSLp1E2 = MSLp1E2 / train_count

mse2 = mse2 / train_count 
MedAE2 = np.median(mae2)
mae2 = mae2 / train_count

In [ ]:
print('script 2 working time ', skript_time2, 'mae', mae2, 'mse ', mse2, 'r2_score ', r2_score2, 'MedAE ', MedAE2, 'MSLp1E ', MSLp1E2)

# Засекаем время выполнения скрипта #3

In [ ]:
start_time = datetime.now()
from catboost import CatBoostClassifier
model3 = CatBoostClassifier(
    #thread_count=2,
    #iterations=5,
    #random_seed=1136926949945377,
)
model3.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation)
    #,logging_level='Silent'
)
prediction = model3.predict_proba(X) # передача тестовой выборки в модель
skript_time3 = datetime.now() - start_time
mae3 = 0
mse3 = 0
ss_tot3 = 0
MSLp1E3= 0
mean_true3 = sum(train_df['target'])/train_count
for n in range(train_count):
    if train_df['target'][n]==0:
        mae3 = mae3 + abs(train_df['target'][n] - prediction[n][0])
        mse3 = mse3 + (train_df['target'][n] - prediction[n][0])**2
        ss_tot3 = ss_tot3 + (prediction[n][0] - mean_true3)**2
        MSLp1E3 = MSLp1E3 + (np.log1p(prediction[n][0]) - np.log1p(train_df['target'][n])) ** 2
    else:
        mae3 = mae3 + abs(train_df['target'][n] - prediction[n][1])
        mse3 = mse3 + (train_df['target'][n] - prediction[n][1])**2
        ss_tot3 = ss_tot3 + (prediction[n][1] - mean_true3)**2
        MSLp1E3 = MSLp1E3 + (np.log1p(prediction[n][1]) - np.log1p(train_df['target'][n])) ** 2
r2_score3 = 1 - (mse3 / train_count)/(ss_tot3 / train_count)
MSLp1E3 = MSLp1E3 / train_count

mse3 = mse3 / train_count 
MedAE3 = np.median(mae3)
mae3 = mae3 / train_count

In [ ]:
print('script 3 working time ', skript_time3, 'mae  ', mae3, 'mse ', mse3, 'r2_score ', r2_score3, 'MedAE ', MedAE3, 'MSLp1E ', MSLp1E3)

# Сравнение результатов 1-ого метода

In [ ]:
print('                     ', 'model #1', '                    ' , 'model #2', '                    model #3')
print('working time:        ',  skript_time1,'              ' , skript_time2, '             ' , skript_time3)
print('mae:                 ',mae,'          ' , mae2, '         ' , mae3)
print('mse:                 ',mse,'          ' , mse2, '         ' ,mse3)
print('r2_score:           ',r2_score,'         ' , r2_score2, '        ' ,r2_score3)
print('MedAE:               ',MedAE,'           ' , MedAE2, '         ' ,MedAE3)
print('MSLp1E:              ',MSLp1E,'         ' , MSLp1E2, '         ' ,MSLp1E3)

# Метод 2: CTR

In [ ]:
train_with_help3_df = pd.read_csv('C:/Users/Анастасия/Desktop/юфу/учеба 4 курс/диплом/yandex_zen_ml_comp_19/train_with_help4.csv', sep=';', engine='python')
test_with_help3_df = pd.read_csv('C:/Users/Анастасия/Desktop/юфу/учеба 4 курс/диплом/yandex_zen_ml_comp_19/test_with_help4.csv', sep=';', engine='python')
train_with_help3_df[sample_id, item, publisher, user, topic_0, weight_0, target, ctr].head()

In [ ]:
y_with_help3 = train_with_help3_df.target
X_with_help3 = train_with_help3_df.drop('target', axis=1)

cat_features_with_help3 = range(0, X_with_help3.shape[1])
print(cat_features_with_help3)

from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X_with_help3, y_with_help3, train_size=0.8, random_state=1234)

start_time = datetime.now()
from catboost import CatBoostClassifier
model_with_help3 = CatBoostClassifier(
    eval_metric='AUC',
    learning_rate=0.8,
    iterations=500,
    random_seed=42,
    od_type='Iter',
    od_wait=20
)
model_with_help3.fit(
    X_train, y_train,
    eval_set=(X_validation, y_validation),
    cat_features=cat_features,
    logging_level='Silent',
    plot=True
)
prediction = model_with_help3.predict_proba(X_with_help3) # передача тестовой выборки в модель
skript_time_with_help3 = datetime.now() - start_time
mae21 = 0
mse21 = 0
ss_tot21 = 0
MSLp1E21 = 0
mean_true21 = sum(train_df['target'])/train_count
res = 0
for n in range(train_count):
    if train_df['target'][n]==0:
        mae21 = mae21 + abs(train_df['target'][n] - prediction[n][0])
        mse21 = mse21 + (train_df['target'][n] - prediction[n][0])**2
        ss_tot21 = ss_tot21 + (prediction[n][0] - mean_true21)**2
        MSLp1E21 = MSLp1E21 + (np.log1p(prediction[n][0]) - np.log1p(train_df['target'][n])) ** 2
        if prediction[n][0]>=0.5:
            res = res + 1
    else:
        mae21 = mae21 + abs(train_df['target'][n] - prediction[n][1])
        mse21 = mse21 + (train_df['target'][n] - prediction[n][1])**2
        ss_tot21 = ss_tot21 + (prediction[n][1] - mean_true21)**2
        MSLp1E21 = MSLp1E21 + (np.log1p(prediction[n][1]) - np.log1p(train_df['target'][n])) ** 2
            if prediction[n][1]>=0.5:
                res = res + 1
        
res=res/train_count
print('2.1     ', res)
r2_score21 = 1 - (mse21/ train_count)/(ss_tot21 / train_count)
MSLp1E21 = MSLp1E21 / train_count

mse21 = mse21 / train_count 
MedAE21 = np.median(mae21)
mae21 = mae21 / train_count

In [ ]:
y_with_help3 = train_with_help3_df.target
X_with_help3 = train_with_help3_df.drop('target', axis=1)

cat_features_with_help3 = range(0, X_with_help3.shape[1])
print(cat_features_with_help3)

from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X_with_help3, y_with_help3, train_size=0.8, random_state=1234)

start_time = datetime.now()
from catboost import CatBoostClassifier
model_with_help4 = CatBoostClassifier(
    thread_count=2,
    iterations=300,
    learning_rate=0.1,
    random_seed=63,
    custom_loss=['AUC', 'Accuracy'],
    use_best_model=True
)
model_with_help4.fit(
    X_train, y_train,
    cat_features=cat_features_with_help3,
    eval_set=(X_validation, y_validation),
    logging_level='Silent',
    plot=True
)
prediction = model_with_help4.predict_proba(X_with_help3) # передача тестовой выборки в модель
skript_with_help4 = datetime.now() - start_time
mae22 = 0
mse22 = 0
ss_tot22 = 0
MSLp1E22 = 0
mean_true22 = sum(train_df['target'])/train_count
res = 0
for n in range(train_count):
    if train_df['target'][n]==0:
        mae22 = mae22 + abs(train_df['target'][n] - prediction[n][0])
        mse22 = mse22 + (train_df['target'][n] - prediction[n][0])**2
        ss_tot22 = ss_tot22 + (prediction[n][0] - mean_true22)**2
        MSLp1E22 = MSLp1E22 + (np.log1p(prediction[n][0]) - np.log1p(train_df['target'][n])) ** 2
        if prediction[n][0]>=0.5:
            res = res + 1
    else:
        mae22 = mae22 + abs(train_df['target'][n] - prediction[n][1])
        mse22 = mse22 + (train_df['target'][n] - prediction[n][1])**2
        ss_tot22 = ss_tot22 + (prediction[n][1] - mean_true22)**2
        MSLp1E22 = MSLp1E22 + (np.log1p(prediction[n][1]) - np.log1p(train_df['target'][n])) ** 2
        if prediction[n][1]>=0.5:
            res = res + 1
              
res=res/train_count
print('2.2     ',res)
        
        
r2_score22 = 1 - (mse22 / train_count)/(ss_tot22 / train_count)
MSLp1E22 = MSLp1E22 / train_count

mse22 = mse22 / train_count 
MedAE22 = np.median(mae22)
mae22 = mae22 / train_count